# Polynomials

Work-In-Progress: This is an experimental notebook

In [1]:
from polynomials import *

There are only two classes involved here: **Term** and **Poly**.

A **Poly** is a list of **Term**.

**Example**:

$-2 -4x +7x^2 -3x^4$ is a polynomial that consists of four terms,

where each term has a coefficient, order, and the same variable:

$-2x^0$ and $-4x^1$ and $+7x^2$ and $-3x^4$.

## Terms

A ``Term`` consists of:

* **coefficient** (``int`` or ``float``)
* **order** (non-negative ``int``)
* **varname** (``string`` variable name)

The term $-2y^3$ is represented as follows:

In [2]:
t1 = Term(-2, 3, 'y')
print(t1)
t1

-2y^3


Term(-2, 3, y)

If no variable name is entered, then the default is **'x'**.

The default variable name is not output in the print representation.

In [3]:
t2 = Term(1, 2)
print(t2)
t2

+1x^2


Term(1, 2)

### Term: Equality

The method, ``unpack``, returns the three quantities that make up a Term.  It's used below to create a copy of a Term and then check for equality.

In [4]:
coef, ord, var = t1.unpack()
t1b = Term(coef, ord, var)
print(f"t1  = {t1}")
print(f"t1b = {t1b}")
print(f"t1 == t1b ? {t1 == t1b}")

t1  = -2y^3
t1b = -2y^3
t1 == t1b ? True


The method, ``copy``, will create an exact copy of a Term.

In [5]:
t1c = t1.copy()
print(f"t1 == t1c ? {t1 == t1c}")
print(f"t1  object ID: {id(t1)}")
print(f"t1c object ID: {id(t1c)}")

t1 == t1c ? True
t1  object ID: 140675219555344
t1c object ID: 140675219554704


### Term: Power

Terms can be raised to a non-negative integer power.

In [6]:
t3a = t1**2
print(t3a)
t3a

+4y^6


Term(4, 6, y)

In [7]:
t3b = t1**0
print(t3b)
t3b

+1


Term(1, 0, y)

### Term: Multiplication

Terms can be multiplied by an ``int`` or a ``float``, on the left or the right.

In [8]:
num = 5.1

In [9]:
t4 = num * t3a
print(t4)
t4

+20.4y^6


Term(20.4, 6, y)

In [10]:
num * t3a == t3a * num

True

### Term: Like terms

Like terms have the same variable and the same order.

e.g., $-2x^7$ and $8x^7$ are like terms.
<p>$-2x^7$ and $8x^3$ are **not** like terms.</p>

In [11]:
t5 = Term(-2, 7)
t6 = Term( 8, 7)
t7 = Term( 8, 3)

In [12]:
t6.like_term(t5)

True

In [13]:
t6.like_term(t7)

False

### Term: Call (variable substitution)

Terms are callable.

Example:

If $t_1 = -2y^3$ and $y=5$,

then $t_1(5) = -2(5^3) = -250$

In [14]:
print(t1)
print(t1(5))

-2y^3
-250


A Term can be called (evaluated) at another Term.

Effectively, this allows for **variable substitution**.

**Example**:

If $t_1 = -2y^3$ and $t_2 = x^2$,

then $t_1(t_2) = -2(x^2)^3 = -2x^6$

and $t_2(t_1) = (-2y^3)^2 = 4y^6$

In [15]:
print(t1)
print(t2)

print(t1(t2))
print(t2(t1))

-2y^3
+1x^2
-2x^6
+4y^6


## Polynomial Basics

A polynomial, ``Poly``, can be constructed from a 1D array of integers or floats that represent coefficients, where the index of a coefficient in the array is its order in the polynomial.

Other methods of polynomial creation are described farther below.

In [16]:
coeffs = [-2, -4, 7, 0, -3]

p = Poly(coeffs)
p

Poly([-2, -4, 7, 0, -3])

By default, 'x' is assumed to be the variable in a polynomial.

In [17]:
print(p)

-2 -4x +7x^2 -3x^4


A ``Poly`` is callable.

In [18]:
a = 3  # A test value

In [19]:
p(a)

-194

Here's a check of the arithmetic in the call made, above.

In [20]:
print(-2 - 4*a + 7*a**2 - 3*a**4)

-194


Internally, a ``Poly`` is made up of a list of ``Terms`` and a ``varname`` (variable name string).

A polynomial's term list is immutable, so it is a *property*; however, ``varname`` is a method, because it can be used to change the polynomials variable (e.g., 'x' -> 'y')

In [21]:
p.terms

[Term(-2, 0), Term(-4, 1), Term(7, 2), Term(-3, 4)]

In [22]:
p.varname()

'x'

A ``Term`` can be retrieved, by its order, using the polynomial's ``term`` method.

In [23]:
t2 = p.term(2)
t2

Term(7, 2)

A ``Term`` consists of a *coefficient*, an *order*, and a *varname*.

In [24]:
t2.coefficient

7

In [25]:
t2.order

2

Terms have a human-readable print representation.

In [26]:
print(t2)

+7x^2


By default, terms use 'x' as a variable name.

In [27]:
t2.varname()

'x'

Here's an example of a polynomial that doesn't use the default *varname* of 'x'.

In [28]:
py = Poly(coeffs, 'y')
py

Poly([-2, -4, 7, 0, -3], 'y')

In [29]:
print(py)

-2 -4y +7y^2 -3y^4


In [30]:
py(3)

-194

In [31]:
py.terms

[Term(-2, 0, y), Term(-4, 1, y), Term(7, 2, y), Term(-3, 4, y)]

We can check if a term is *constant*, *linear*, or *quadratic*.  If it is, then the coeffient of the term is returned, otherwise ``False`` is returned.

In [32]:
py.terms[0].is_linear()

False

In [33]:
py.terms[0].is_constant()

-2

In [34]:
py.terms[2].is_quadratic()

7

## Other Ways to Construct Polynomials

### From a String Representation

A ``Poly`` can also be constructed from a string representation of a polynomial, such as "-2 -4x +7x^2 -3x^4", where...

* there **must** be a space between each term,
* and, except for the first term, every term must begin with a + or - sign, and then a number, unless the number is a 1.

In [35]:
polystr = "-2 -4x +7x^2 -3x^4"

p2 = Poly(polystr)
print(p2)
p2

-2 -4x +7x^2 -3x^4


Poly([-2, -4, 7, 0, -3])

In [36]:
polystry = "7 +3y -y^2 -9y^3"

p2y = Poly(polystry)

In [37]:
p2y.terms

[Term(7, 0, y), Term(3, 1, y), Term(-1, 2, y), Term(-9, 3, y)]

In [38]:
print(p2y)
p2y

+7 +3y -1y^2 -9y^3


Poly([7, 3, -1, -9], 'y')

### From a List of Pairs (Tuples)

In [39]:
coef_order_pairs_a = [(-2,0), (-4,1), (7, 2), (-3, 4)]

p3a = Poly(coef_order_pairs_a)
print(p3a)
p3a

-2 -4x +7x^2 -3x^4


Poly([-2, -4, 7, 0, -3])

### From a List of Pairs (Lists)

In [40]:
coef_order_pairs_b = [[-2,0], [-4,1], [7, 2], [-3, 4]]

p3b = Poly(coef_order_pairs_b)
print(p3b)
p3b

-2 -4x +7x^2 -3x^4


Poly([-2, -4, 7, 0, -3])

### From a List of Terms

In [41]:
list_of_terms = [Term(-2,0), Term(-4,1), Term(7, 2), Term(-3, 4)]

p4 = Poly(list_of_terms)
print(p4)
p4

-2 -4x +7x^2 -3x^4


Poly([-2, -4, 7, 0, -3])

### From a List of Roots

In [42]:
roots = [1, 2, 3]

p5 = from_roots(roots)
print(p5)
p5

+6 -11x +6x^2 -1x^3


Poly([6, -11, 6, -1])

## Combining Like Terms

Like terms (i.e., same order terms) are automatically combined during various polynomial operations, include the construction of polynomials.

In [43]:
test_terms = [Term(-2,0), Term(-4,1), Term(-1,0), Term(7,2), Term(-3,4), Term(-2,2)]

In [44]:
print(test_terms)
print(combine_like_terms(test_terms))

[Term(-2, 0), Term(-4, 1), Term(-1, 0), Term(7, 2), Term(-3, 4), Term(-2, 2)]
[Term(-3, 0), Term(-4, 1), Term(5, 2), Term(-3, 4)]


In [45]:
polystr2 = "-2 -4x -1 +7x^2 -3x^4 -2x^2"
p2 = Poly(polystr2)
print(p2)
p2

-3 -4x +5x^2 -3x^4


Poly([-3, -4, 5, 0, -3])

In [46]:
polystr3 = "-2 +3x^4 -4x +2 +7x^2 +4x -3x^4 -7x^2"
p3 = Poly(polystr3)
print(p3)
p3

0


Poly([0])

## Testing Some Polynomial Strings

In [47]:
polystr = "-2 -4x^1 +7x^2 -3x^4"  # Added '^1' to linear term

p = Poly(polystr)
print(p)
p.terms

-2 -4x +7x^2 -3x^4


[Term(-2, 0), Term(-4, 1), Term(7, 2), Term(-3, 4)]

In [48]:
polystr = "-2x^0 -4x^1 +7x^2 -3x^4"  # Added 'x^0' to constant term

p = Poly(polystr)
print(p)
p.terms

-2 -4x +7x^2 -3x^4


[Term(-2, 0), Term(-4, 1), Term(7, 2), Term(-3, 4)]

In [49]:
polystr = "-4x -2 -3x^4 +7x^2"  #  Terms not in order 

p = Poly(polystr)
print(p)
p.terms

-2 -4x +7x^2 -3x^4


[Term(-2, 0), Term(-4, 1), Term(7, 2), Term(-3, 4)]

In [50]:
polystr = "2 -x -3 +7x^2 -3x^4"  # No coefficient on linear term, 'x'

p = Poly(polystr)
print(p)
p.terms

-1 -1x +7x^2 -3x^4


[Term(-1, 0), Term(-1, 1), Term(7, 2), Term(-3, 4)]

In [51]:
polystr = "2 -x -3 +7x^2 +x -3x^4"  # linear terms cancel

p = Poly(polystr)
print(p)
p.terms

-1 +7x^2 -3x^4


[Term(-1, 0), Term(7, 2), Term(-3, 4)]

In [52]:
polystr = "-4x^2 -x -3 +7x^2 +x -3x^2"  # Quadratic terms collapse and cancel

p = Poly(polystr)
print(p)
p.terms

-3


[Term(-3, 0)]

## Polynomial Arithmetic

Polynomials can be added, subtracted, multiplied, negated, and raised to a positive integer power.

In [53]:
q1 = Poly('1 +2x')
print(f"q1 = {q1}")

q2 = Poly('1 -2x')
print(f"q2 = {q2}")

q1 = +1 +2x
q2 = +1 -2x


In [54]:
print(q1 + 1)
print(1 + q1)

+2 +2x
+2 +2x


In [55]:
print(q1)
print(-q1)
print(3*q1)
print(1 + q1)
print(q1 + 1)
print(1 + -q1)
print(1 - q1)
print(q1 - 1)

+1 +2x
-1 -2x
+3 +6x
+2 +2x
+2 +2x
-2x
+2x
+2x


In [56]:
foo = Poly([2], 'x')
print(foo)

+2


In [57]:
foo.terms

[Term(2, 0)]

In [58]:
print(f"({q1}) + ({q2}) = {q1 + q2}")

(+1 +2x) + (+1 -2x) = +2


In [59]:
print(f"({q1}) - ({q2}) = {q1 - q2}")

(+1 +2x) - (+1 -2x) = +4x


In [60]:
print(f"({q1}) * ({q2}) = {q1 * q2}")

(+1 +2x) * (+1 -2x) = +1 -4x^2


In [61]:
print(f"({q1}) * ({q1}) = {q1 * q1}")

(+1 +2x) * (+1 +2x) = +1 +4x +4x^2


In [62]:
print(f"({q2}) * ({q2}) = {q2 * q2}")

(+1 -2x) * (+1 -2x) = +1 -4x +4x^2


In [63]:
print(f"-({q1}) = {-q1}")

-(+1 +2x) = -1 -2x


In [64]:
print(f"-({q2}) = {-q2}")

-(+1 -2x) = -1 +2x


In [65]:
print(f"({q1})**2 = {q1**2}")

(+1 +2x)**2 = +1 +4x +4x^2


In [66]:
print(f"({q2})**2 = {q2**2}")

(+1 -2x)**2 = +1 -4x +4x^2


## Polynomial Representation

The representation of a polynomial changes between two options, depending the estimated length of the representation.  The representation heuristic tries to produce the shortest representation possible, while still allow it to be one that can be used to recreate the polynomial.

In [67]:
coeffs1 = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7]
p_large_order1 = Poly(coeffs1)
p_large_order1

Poly([(1, 0), (7, 19)])

In [68]:
coeffs2 = [1, 0, 0, 0, 0, 0, 7]
p_large_order2 = Poly(coeffs2)
p_large_order2

Poly([(1, 0), (7, 6)])

In [69]:
coeffs3 = [1, 0, 0, 0, 0, 7]
p_large_order3 = Poly(coeffs3)
p_large_order3

Poly([1, 0, 0, 0, 0, 7])

## Using Floats

Polynomial coefficients can be floats, as well as integers, or a mix of the two.

In [70]:
coeffs = [-2.2, -4, 7.3, -0.0, -3]

p1 = Poly(coeffs)
print(p1)
p1

-2.2 -4x +7.3x^2 -3x^4


Poly([-2.2, -4, 7.3, 0, -3])

In [71]:
p2 = Poly('-2.2 -4x +7.3x^2 -3x^4')
print(p2)
p2

-2.2 -4x +7.3x^2 -3x^4


Poly([-2.2, -4, 7.3, 0, -3])

## Polynomial Derivatives and Antiderivatives

In [72]:
print(p1)
p1_der = p1.derivative()
print(p1_der)

-2.2 -4x +7.3x^2 -3x^4
-4 +14.6x -12x^3


In [73]:
p1_der_antider = p1_der.antiderivative(111)
print(p1_der_antider)

+111 -4.0x +7.3x^2 -3.0x^4


## Scratchwork

In [74]:
r = Poly('-3 +2x +7x^2')
print(f"r = {r}")

r = -3 +2x +7x^2


In [75]:
s = Poly('5 +4y', 'y')
print(f"s = {s}")

s = +5 +4y


In [76]:
print(2*r)

-6 +4x +14x^2


In [77]:
print(r*2)

-6 +4x +14x^2


In [78]:
print(s(r))

-7 +8x +28x^2


In [79]:
# print(r(s))  # ValueError: Mult: Variables must be the same, x != y

# NumPy Polynomials

This is a better library for polynomials.

In [80]:
from numpy.polynomial import Polynomial

poly_coeff = [-2, -4, 7, 0, -3]
poly = Polynomial(poly_coeff)
poly

Polynomial([-2., -4.,  7.,  0., -3.], domain=[-1,  1], window=[-1,  1])

In [81]:
poly(3)

-194.0

In [82]:
print(poly)

-2.0 - 4.0·x¹ + 7.0·x² + 0.0·x³ - 3.0·x⁴


In [87]:
poly1 = Polynomial([1, 1])
poly1

Polynomial([1., 1.], domain=[-1,  1], window=[-1,  1])

In [84]:
poly2 = Polynomial([1, -1])
poly2

Polynomial([ 1., -1.], domain=[-1,  1], window=[-1,  1])

In [85]:
poly1 * poly2

Polynomial([ 1.,  0., -1.], domain=[-1.,  1.], window=[-1.,  1.])

In [86]:
# help(Polynomial)